# GEP-OnSSET GIS-Extraction Notebook for GEP-OnSSET

This is the GEP-OnSSET GIS extraction notebook that runs in bulk. 
First, browse and select each layer, then run the processing for all layers.

You may start by selecting this cell, then in the menu click **Run > Run All Cells**

### Useful hints and common error messages
* Make sure that all input layers are using EPSG:4326 as the coordinate system
* Make sure that the target "crs" is in a coordinate system using meters as the unit
* It is often useful to clip all the input layers to the country boundaries in order to reduce processing times
* Make sure that each dataset actually has some data within the country boundaries
* Some of the datasets require the user to choose values from a dropdown list below
* For hydro points and mini-grids, the vector layers need some specific column names to work
* In case a dataset still does not work, try opening it in QGIS and run the *Fix geometries* tool and save the new layer.
* If things do not work, it may be useful to go to the very top of this Jupyter Notebook and start again from cell 1

## Importing necessary packages (Mandatory)

Packages to be used are imported from the funcs.ipynb.

In [ ]:
%run funcs.ipynb
import traceback
import time
#import warnings

#warnings.filterwarnings("ignore")

# Step 1: Indicate the layers and parameters to be used

First, define the coordinate system (crs). 
Then select the correct layer each time. If there is a layer you do not have or wish to use, press **Cancel**

In [ ]:
crs = 'EPSG:3395'

In [ ]:
messagebox.showinfo('OnSSET extraction', 'Output folder')
workspace = filedialog.askdirectory()

In [ ]:
messagebox.showinfo('OnSSET', 'Select the admin boundaries')
admin = filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the clusters')
clusters_path = filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*")))
clusters = gpd.read_file(clusters_path)
options = clusters.columns.tolist()
messagebox.showinfo('OnSSET', 'Select the column with population counts in the clusters')
x = dropdown_popup(options)
print('Population column: ' + x)

In [ ]:
## Raster layers
messagebox.showinfo('OnSSET', 'Select the Solar GHI layer')
ghi_layer = filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Travel Time layer')
travel_layer = filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Elevation layer')
elevation_layer = filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Land Cover layer')
lc_layer = filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Wind layer')
wind_layer = filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Night Lights layer')
ntl_layer = filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Custom Demand layer')
custDem_layer = filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Substations layer')
sub_layer = filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Existing HV layer')
HVexist_layer = filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Planned HV layer')
HVplan_layer = filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Existing MV layer')
MVexist_layer = filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Planned MV layer')
MVplan_layer = filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Roads layer')
road_layer = filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Distribution Transformer layer')
trx_layer = filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Hydro layer')
hydro_layer = filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*")))
hydro=gpd.read_file(hydro_layer)

messagebox.showinfo('OnSSET', 'Select the column which describes the hydropower POWER potential in each location')
options = hydro.columns.tolist()
hydropower = dropdown_popup(options)

messagebox.showinfo('OnSSET', 'Select the UNIT of the power potential')
options=['W', 'kW', 'MW']
hydrounit = dropdown_popup(options)
print(hydropower)
print(hydrounit)

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Mini Grid layer')
exist_MG_layer = filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*")))

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Admin 1 layer')
adm1_layer = filedialog.askopenfilename(filetypes = (("vector",["*.shp", "*.gpkg", "*.geojson"]),("all files","*.*")))

if adm1_layer != '':
    admin_1 = gpd.read_file(adm1_layer)
    messagebox.showinfo('OnSSET', 'Select the column which contains the Admin 1 level names')
    options = admin_1.columns.tolist()
    admin_col_name = dropdown_popup(options)

# Step 2: Process layers

## Import admin

In [ ]:
admin = gpd.read_file(admin)

## Extract Global Horizontal Irradiation (GHI) from Raster layer

In [ ]:
out, ghi_path = zonal_stat_bulk('GHI', ghi_layer, clusters, 'mean')
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Extract Travel Time from Raster layer

In [ ]:
out, travel_path = zonal_stat_bulk('TravelTime', travel_layer, clusters, 'mean')
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Extract Land Cover from Raster layer

In [ ]:
out, lc_path = zonal_stat_bulk('LandCover', lc_layer, clusters, 'mode')
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Extract Elevation and Slope from Raster layer

In [ ]:
try:
    clusters = processing_elevation_and_slope_bulk(elevation_layer, "Elevation","mean",clusters, workspace,crs)
except Error as e:
    print('Could not process Elevation, or layer was not selected')
    print(e)

## Extract Wind Velocity from Raster layer

In [ ]:
out, wind_path = zonal_stat_bulk('WindVel', wind_layer, clusters, 'mean')
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Extract Night Lights from Raster layer

In [ ]:
out, ntl_path = zonal_stat_bulk('NightLight', ntl_layer, clusters, 'mean')
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Extract Custom Demand from Raster layer

In [ ]:
out, cd_path = zonal_stat_bulk('CustomDemand', custDem_layer, clusters, 'mean')
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Preparing to run the vector data

In [ ]:
clusters = preparing_for_vectors(workspace, clusters, crs)

## Extract Distance from Substations (Vector point layer)

In [ ]:
out, substation_path = processing_points_bulk("Substation", sub_layer, admin, crs, workspace, clusters, mg_filter=False)
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Extract Distance from Existing high voltage lines (Vector line layer)

In [ ]:
out, existing_hv_path = processing_lines_bulk("Existing_HV", HVexist_layer, admin, crs, workspace, clusters)
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Extract Distance from Planned high voltage lines (Vector line layer)

In [ ]:
out, planned_hv_path = processing_lines_bulk("Planned_HV", HVplan_layer, admin, crs, workspace, clusters)
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Extract Distance from Existing medium voltage lines (Vector line layer) 

In [ ]:
out, existing_mv_path = processing_lines_bulk("Existing_MV", MVexist_layer, admin, crs, workspace, clusters)
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Extract Distance from Planned medium voltage lines (Vector line layer)

In [ ]:
out, planned_mv_path = processing_lines_bulk("Planned_MV", MVplan_layer, admin, crs, workspace, clusters)
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Extract Distance from Roads (Vector line layer)

In [ ]:
out, road_path = processing_lines_bulk("Road", road_layer, admin, crs, workspace, clusters)
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Extract Distance from Transformers (Vector point layer)

In [ ]:
out, trx_path = processing_points_bulk("Service Transformer", trx_layer, admin, crs, workspace, clusters, mg_filter=False)
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Extract Distance from hydro points (Vector point layer)

In [ ]:
out = hydro_bulk(admin, hydro, hydropower, hydrounit, crs, workspace, clusters)
if type(out) == gpd.geodataframe.GeoDataFrame:
    clusters = out

## Adding admin 1 name to clusters (Vector polygon layer)


In [ ]:
if adm1_layer != '':
    out = admin_1_bulk("Admin_1", admin_1, admin_col_name, crs, workspace, clusters)
    if type(out) == gpd.geodataframe.GeoDataFrame:
        clusters = out

## Conditioning & Export (Mandatory)

This is the final cell in the extraction. This cell has to be run.

In [ ]:
clusters = conditioning(clusters, workspace, x)
print('Workspace: ', workspace)